# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key




### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [9]:
file = ("../output_data/describe_data.csv")
file_df = pd.read_csv(file)
file_df.head()


,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ostrovnoy,68.05,39.51,29.39,97,98,5.75,RU,1603330459
1,Mahébourg,-20.41,57.70,73.40,78,40,10.29,MU,1603330460
2,Qaanaaq,77.48,-69.36,20.17,85,100,6.58,GL,1603330460
3,Zhuhai,22.28,113.57,73.40,69,20,23.04,CN,1603330461
4,Cape Town,-33.93,18.42,59.00,87,0,14.99,ZA,1603330461


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
gmaps.configure(api_key=g_key)
hum = file_df["Humidity"].astype(float)
hum_max = hum.max()
location = file_df[["Lat", "lng"]]

fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(location, weights = hum, dissipating=False, max_intensity=hum_max, point_radius=3)
fig.add_layer(heat_layer)
fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
# Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API 
# requests to a reasonable number.
new_data = file_df.loc[(file_df["Wind Speed"] <= 10) & (file_df["Cloudiness"] ==0) & (file_df["Max Temp"] >= 70)
                       & (file_df["Max Temp"] < 80) ]
              
new_data = new_data.dropna(how='any')

new_data.head()


,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
26,Colonelganj,27.13,81.70,74.97,62,0,2.89,IN,1603330470
93,Antakya,36.21,36.16,73.40,64,0,5.82,TR,1603330547
152,Sinnamary,5.38,-52.95,79.75,83,0,9.75,GF,1603330460
157,Charlestown,-32.95,151.67,73.99,69,0,7.61,AU,1603330575
180,Le Marin,14.47,-60.87,77.00,88,0,2.24,MQ,1603330585


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_df = new_data.loc[:,['City', 'Country', 'Lat', 'lng']]

hotel_df['Hotel Name'] = ""

hotel_df.head()



,City,Country,Lat,lng,Hotel Name
26,Colonelganj,IN,27.13,81.70,
93,Antakya,TR,36.21,36.16,
152,Sinnamary,GF,5.38,-52.95,
157,Charlestown,AU,-32.95,151.67,
180,Le Marin,MQ,14.47,-60.87,


In [13]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel", 
         'keyword' : 'hotel',
         'radius': 5000,
         'key' : g_key}


In [14]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row['lng']
    city_name = row['City']
    
    params["location"] = f"{lat},{lng}"
    print(f"Retrieving Results for index {index}:{city_name}.")
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing filed/result .. skipping.")
        
    print("-------------------------------")
    
print("End")

Retrieving Results for index 26:Colonelganj.
Closest hotel in Colonelganj is Anand Hotel.
-------------------------------
Retrieving Results for index 93:Antakya.
Closest hotel in Antakya is Shahut Hotel.
-------------------------------
Retrieving Results for index 152:Sinnamary.
Closest hotel in Sinnamary is L'oiseau de Paradis.
-------------------------------
Retrieving Results for index 157:Charlestown.
Closest hotel in Charlestown is Mercure Charlestown.
-------------------------------
Retrieving Results for index 180:Le Marin.
Closest hotel in Le Marin is Club Med Les Boucaniers - Martinique.
-------------------------------
Retrieving Results for index 260:Pahrump.
Closest hotel in Pahrump is Pahrump Nugget.
-------------------------------
Retrieving Results for index 278:Cayenne.
Closest hotel in Cayenne is Royal Amazonia.
-------------------------------
Retrieving Results for index 280:Santa Inês.
Closest hotel in Santa Inês is Monterrey Hotel.
-------------------------------
Re

In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "lng"]]

In [16]:
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)

fig


Figure(layout=FigureLayout(height='420px'))